# 행정동 경제지표 데이터 전처리 및 합병

### 사용 라이브러리

In [1]:
# 데이터 처리 라이브러리
import requests, json, io
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

# Dask 라이브러리
import dask
from dask import delayed
import dask.dataframe as dd
from dask.distributed import Client, progress

In [2]:
# Parallel processing을 위한 DASK 클라이언트 설정
client = Client(threads_per_worker=2, n_workers=5)  # 사용 코어 수에 따라 조정
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 15.42 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53521,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 15.42 GiB
Comm: tcp://127.0.0.1:53548,Total threads: 2
Dashboard: http://127.0.0.1:53556/status,Memory: 3.08 GiB
Nanny: tcp://127.0.0.1:53524,


## 행정동 데이터 매핑
차후 추가해줄 행정동 데이터를 위해 법정동코드를 사용해 행정동 구분을 매핑해 주도록 하겠습니다.

In [10]:
# 데이터 불러오기
df = pd.read_parquet('./data/jeonnam_clustered.parquet')

In [11]:
# 법정동 코드와 행정동 코드 매핑
mapping_df = pd.read_csv('./data/법정동_행정동_매핑.csv', encoding='utf-8-sig', usecols=['LGDNG_CD', 'ADSTRD_NM', 'ADSTRD_CD'])

# 컬럼명 변경
mapping_df.columns = ['법정동코드', '행정읍면동명', '행정동코드']

In [12]:
# 법정동 코드를 기준으로 데이터 병합
df_mapped = pd.merge(df, mapping_df, on='법정동코드', how='left')

In [21]:
# address 컬럼 기준으로 중복된 데이터가 있다면 1:1 매핑이 안된 것으로 간주
df_mapped['매핑실패'] = df_mapped.duplicated(subset='address', keep=False)

df_mapped['매핑실패'].sum()

5332

In [22]:
# 행정동 분류가 실패한 데이터만 추출
df_needs_remapping = df_mapped[df_mapped['매핑실패']][['address', 'mean_longitude', 'mean_latitude', '행정읍면동명', '매핑실패']]

---
### 위,경도를 사용한 2차 매핑
매핑 실패한 데이터에 대해선 위도, 경도를 사용하여 행정동 정보를 매핑하겠습니다.

In [16]:
# 카카오API를 사용하여 좌표-> 행정동 주소로 변환하는 함수
# 많은 양의 데이터를 처리하기 때문에 @delayed 데코레이터를 사용하여 지연된 함수로 만듦
@delayed
def reverse_geocoding(lat, long):
    '''역 지오코딩하기
    주소의 위경도를 넣으면 행정동 주소 출력 '''
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x="+str(long)+"&y="+str(lat)
    headers = {"Authorization": "KakaoAK b39264b82c3bb54a44fb499ffe2435ed"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    adm_address = full_address['documents'][1]['address_name']
    return adm_address

In [23]:
# 데이터프레임에서 위도 경도 추출
lat = df_needs_remapping['mean_latitude'].reset_index(drop=True)
long = df_needs_remapping['mean_longitude'].reset_index(drop=True)

# 위도 경도를 이용하여 행정동 주소로 변환
# dask.delayed를 사용하여 함수를 지연된 함수로 만들어 병렬처리
lazy_results = []
for i in range(len(lat)):
    lazy_results.append(reverse_geocoding(lat[i], long[i]))

In [24]:
# Dask를 사용하여 병렬로 처리 
# dask_output = dask.compute(*lazy_results)  # (시간이 너무 오래 걸려서 주석 처리)

In [26]:
# 결과를 데이터프레임에 저장
df_needs_remapping['행정동주소'] = dask_output

In [27]:
# 행정동 주소에서 읍면동만 추출
df_needs_remapping['행정읍면동명'] = df_needs_remapping['행정동주소'].str.split(' ').str[2]

In [33]:
# df_needs_remapping 데이터프레임을 기존 df_mapped 데이터프레임에 병합
df_remapped = pd.merge(df_mapped, df_needs_remapping[['address', '행정읍면동명']], on='address', how='left')
df_remapped['행정읍면동명'] = df_remapped['행정읍면동명_y'].fillna(df_remapped['행정읍면동명_x'])
df_remapped.drop(['행정읍면동명_x', '행정읍면동명_y', '매핑실패'], axis=1, inplace=True)

In [34]:
# 매핑 오류로 인해 반복된 데이터는 제거
df_remapped.drop_duplicates(subset='address', keep='first', inplace=True)

In [57]:
# 데이터 저장
df_remapped.to_parquet('./data/jeonnam_clustered_mapped_ADMadd.parquet')

In [36]:
# dask 클라이언트 종료
client.close()

---
*DASK를 이용한 매핑과정을 건너 뛰려면 여기서부터 실행*
## 행정동별 경제지표 데이터 전처리

In [39]:
df_econ = pd.read_csv('./data/전남_산업체별경제지표정보.csv', encoding='cp949')
df_econ.head()

,행정구역(읍면동),산업분류(대분류),항목,단위,2020 년,Unnamed: 5
0,전라남도,전산업,사업체수,NaN,228219,NaN
1,전라남도,전산업,종사자수,NaN,847692,NaN
2,전라남도,전산업,남,NaN,501882,NaN
3,전라남도,전산업,여,NaN,345810,NaN
4,전라남도,전산업,매출액,NaN,231459174,NaN


In [40]:
# 필요없는 컬럼 제거
df_econ.drop(['단위', 'Unnamed: 5'], axis=1, inplace=True)

In [41]:
# 2020년 컬럼을 숫자로 변환, 숫자가 아닌 값은 NaN으로 변환
df_econ['2020 년'] = pd.to_numeric(df_econ['2020 년'], errors='coerce')

In [45]:
# 항목을 컬럼으로 사용
df_econ_pivot = df_econ.pivot_table(index=['행정구역(읍면동)', '산업분류(대분류)'], columns='항목', values='2020 년', aggfunc='sum')
df_econ_pivot.head()

항목                                감가·대손상각비  경상연구개발비  광고선전비    급여총액  기타영업비용  \
행정구역(읍면동) 산업분류(대분류)                                                          
가사문학면     건설업(41~42)                  13.0      0.0    0.0   499.0   718.0   
          공공행정, 국방 및 사회보장 행정(84)    1107.0     55.0   53.0  1412.0  3290.0   
          광업(05~08)                    0.0      0.0    0.0     0.0     0.0   
          교육 서비스업(85)               4848.0      3.0    0.0  7131.0  6262.0   
          금융 및 보험업(64~66)              0.0      0.0    0.0     0.0     0.0   

항목                                   남      매출액  매출원가  사업체수  세금과공과     여  \
행정구역(읍면동) 산업분류(대분류)                                                        
가사문학면     건설업(41~42)              12.0   1392.0   0.0   5.0   25.0   2.0   
          공공행정, 국방 및 사회보장 행정(84)  13.0   6272.0   0.0   3.0   37.0  11.0   
          광업(05~08)                0.0      0.0   0.0   0.0    0.0   0.0   
          교육 서비스업(85)             33.0  23795.0   0.0   4.0   24.0  46.0   
          금융 및 보험업(64~66)          0.0      0.0   0.0   1.0    0.0   0.0   

항목                                   영업비용   영업이익      인건비    임차료  종사자수  
행정구역(읍면동) 산업분류(대분류)                                                     
가사문학면     건설업(41~42)               1278.0  114.0    518.0    4.0  14.0  
          공공행정, 국방 및 사회보장 행정(84)   6272.0    0.0   1672.0   58.0  24.0  
          광업(05~08)                   0.0    0.0      0.0    0.0   0.0  
          교육 서비스업(85)             23787.0    8.0  12541.0  109.0  79.0  
          금융 및 보험업(64~66)             0.0    0.0      0.0    0.0   0.0

각각의 산업을 따로 보기보단 전체적인 경제적 지표를 구하고자 하기 때문에 모든 산업의 합계를 구해주겠다.

In [43]:
# 산업별 합계를 계산
for region in df_econ_pivot.index.levels[0]:
    df_econ_pivot.loc[(region, '합계'), :] = df_econ_pivot.loc[region, :].sum()
    
# 1차 인덱스를 기준으로 정렬
df_econ_pivot = df_econ_pivot.sort_index(level=0)

# 합계 행만 추출
df_econ_pivot_total = df_econ_pivot.loc[(slice(None), '합계'), :]

# 2차 인덱스 (산업 구분) 제거
df_econ_pivot_total.reset_index(level=1, drop=True, inplace=True)

In [46]:
# 행정구역(읍면동)이 "전라남도"인 데이터 제외
df_econ_pivot_total = df_econ_pivot_total[df_econ_pivot_total.index != '전라남도']

In [48]:
# 경제지표 중 원하는 컬럼들만 선택 (종사자수, 사업체수, 급여총액, 매출액, 영업이익)
df_selected = df_econ_pivot_total[['종사자수', '사업체수', '급여총액', '매출액', '영업이익']]

# 인덱스를 컬럼으로 변환
df_selected.reset_index(inplace=True)

In [49]:
# 각 산업별 지표를 종사자수로 나누어 비율로 변환
df_selected['종사자당 사업체수'] = df_selected['사업체수'] / df_selected['종사자수']
df_selected['종사자당 급여총액'] = df_selected['급여총액'] / df_selected['종사자수']
df_selected['종사자당 매출액'] = df_selected['매출액'] / df_selected['종사자수']
df_selected['종사자당 영업이익'] = df_selected['영업이익'] / df_selected['종사자수']

df_selected.head()

항목,행정구역(읍면동),종사자수,사업체수,급여총액,매출액,영업이익,종사자당 사업체수,종사자당 급여총액,종사자당 매출액,종사자당 영업이익
0,가사문학면,845.0,218.0,31559.0,118867.0,10560.0,0.257988,37.347929,140.671006,12.497041
1,간전면,701.0,272.0,16076.0,134719.0,8592.0,0.388017,22.932953,192.181170,12.256776
2,강진읍,14972.0,4870.0,444344.0,3240270.0,662714.0,0.325274,29.678333,216.421988,44.263559
3,겸면,2307.0,412.0,77747.0,473946.0,1244.0,0.178587,33.700477,205.438231,0.539228
4,겸백면,363.0,132.0,7926.0,37248.0,5835.0,0.363636,21.834711,102.611570,16.074380


In [56]:
# 전처리된 경제 데이터 저장
df_selected.to_csv('./data/jeonnam_econ.csv', encoding='cp949', index=False)

---
---